# Evaluation of Extractive Summarization Algos

The goal is to research differences in the summarization efficiency of the popular summarization extractive algorithms with respect to the text language.

#### Languages:
- English
- Ukrainian
- Early New High German (?)

#### Algorithms:
- Random
- Frequency
- LexRank
- TextRank
- Luhn
- LSA (Latent Semantic Analysis)
- KL (Kullback-Leibler)

#### Evaluation approaches:
- SummaQA (https://github.com/ThomasScialom/summa-qa)
- BLANC (https://github.com/PrimerAI/blanc)
- SUPERT (https://github.com/yg211/acl20-ref-free-eval), too much?..
- BLEU (actually it's for MT...)  (**NOT SUITABLE - requires reference human-edited summary**)
- ROUGE (https://pypi.org/project/rouge-score/) (**NOT SUITABLE - requires reference human-edited summary**)


In [258]:
import requests
import json
import math
import numpy as np
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.random import RandomSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.reduction import ReductionSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [259]:
TEXT_COUNT = 100
MAX_TEXT_LENGTH = 512 # for the BERT pretrained model
SUMMARY_SIZE_PERCENTAGE = 40

In [260]:
def get_request_url(lang):
    return f"https://datasets-server.huggingface.co/rows?dataset=wikimedia%2Fwikipedia&config=20231101.{lang}&split=train&offset=0&length={TEXT_COUNT}";


In [261]:
class Languages:
    def __init__(self, en_lang, uk_lang):
        self.en = en_lang
        self.uk = uk_lang

In [307]:
import csv

class LanguageConfig:
    """A config for a single language.

    Attributes:
        text_tuples      Tuples with text data in the form (title, text).
        short            The short name of the language, e.g. "en".
    """
    def __init__(self, short_name, long_name):
        """short name of the language, e.g. \"en\""""
        self.short = short_name
        self.long = long_name
        self.tokenizer = Tokenizer(long_name)
        self.stemmer = Stemmer(long_name)
        self.stop_words = get_stop_words(long_name)
        self.text_tuples = self.get_text_tuples(short_name)
        """tuples with text data in the form (title, text)"""

    def get_text_tuples(self, name):
        response = requests.get(get_request_url(name))
        parsed_response = json.loads(response.text)
        text_tuples = []
        
        for item in parsed_response['rows']:
            tuple = (item['row']['title'], self.shorten_text(item['row']['text']))
            text_tuples.append(tuple)
        return text_tuples

    def shorten_text(self, text):
        sentences = self.tokenizer.to_sentences(text)
        total_word_count = 0
        last_sentence_index = 0
        for index, next in enumerate(sentences):
            words_count = len(self.tokenizer.to_words(next))
            total_word_count += words_count
            if (total_word_count <= MAX_TEXT_LENGTH):
                last_sentence_index = index
            else:
                break
        separator = ' ' if self.short == 'en' else '. '
        return separator.join(sentences[0:last_sentence_index])

In [308]:
languages = Languages(LanguageConfig('en', 'english'),
                      LanguageConfig('uk', 'ukrainian'))

### Generating summaries

In [314]:
class TextWithSummaries:
    def __init__(self, text_tuple, lang_config, writer):
        stemmer = lang_config.stemmer
        tokenizer = lang_config.tokenizer
        stop_words = lang_config.stop_words

        self.title = text_tuple[0]
        self.text = text_tuple[1]
        self.random_summary = self.stringify_summary(self.get_summary(RandomSummarizer(stemmer), tokenizer, stop_words), lang_config.short)
        self.luhn_summary = self.stringify_summary(self.get_summary(LuhnSummarizer(stemmer), tokenizer, stop_words), lang_config.short)
        self.kl_summary = self.stringify_summary(self.get_summary(KLSummarizer(stemmer), tokenizer, stop_words), lang_config.short)
        self.reduction_summary = self.stringify_summary(self.get_summary(ReductionSummarizer(stemmer), tokenizer, stop_words), lang_config.short)
        self.text_rank_summary = self.stringify_summary(self.get_summary(TextRankSummarizer(stemmer), tokenizer, stop_words), lang_config.short)
        self.lex_rank_summary = self.stringify_summary(self.get_summary(LexRankSummarizer(stemmer), tokenizer, stop_words), lang_config.short)
        self.lsa_summary = self.stringify_summary(self.get_summary(LsaSummarizer(stemmer), tokenizer, stop_words), lang_config.short)
        self.persist(writer)
    
    def get_summary(self, summarizer, tokenizer, stop_words):
        parser = PlaintextParser.from_string(self.text, tokenizer)
        summarizer.stop_words = stop_words
        sentences_length = len(tokenizer.to_sentences(self.text))
        sentences_count = math.floor(sentences_length * SUMMARY_SIZE_PERCENTAGE / 100)
        return summarizer(parser.document, sentences_count)

    def stringify_summary(self, summary, lang_name):
        print(summary)
        separator = ' ' if lang_name == 'en' else '. '
        return separator.join(list(map(lambda sent: sent._text, summary)))

    def persist(self, writer):
            writer.writerow([self.title,
                            self.text,
                            self.random_summary,
                            self.luhn_summary,
                            self.kl_summary,
                            self.reduction_summary,
                            self.text_rank_summary,
                            self.lex_rank_summary,
                            self.lsa_summary])
        

In [310]:
def generate_summaries(lang):
    with open(f'./datasets/{lang}.csv', 'w', encoding='UTF8') as f:
        header = ['title',
                  'text',
                  'random_summary',
                  'luhn_summary',
                  'kl_summary',
                  'reduction_summary',
                  'text_rank_summary',
                  'lex_rank_summary',
                  'lsa_summary']
        writer = csv.writer(f)
        writer.writerow(header)
        lang_config = getattr(languages, lang)
        for tuple in lang_config.text_tuples:
            print(f"Generating summaries for {tuple[0]}")
            TextWithSummaries(tuple, lang_config, writer)

In [315]:
# generate_summaries(languages.en.short)
generate_summaries(languages.uk.short)

Generating summaries for Географія
(<Sentence: Геогра́фія, або заст>, <Sentence: В сучасному розумінні поняття «географія>, <Sentence: заміщено поняттям «географічні науки>, <Sentence: Поняття Уперше термін «географія>, <Sentence: Відповідно до об'єктів вивчення в сучасній географії виділяють природничо-географічні і суспільно-географічні науки, які тісно між собою пов'язані спільними завданнями всебічного вивчення природно- і виробничо-територіальних комплексів та їхніх компонентів>, <Sentence: Природничо-географічні (фізико-географічні) науки вивчають закономірності будови й розвитку географічної оболонки та її окремих складових частин (рельєф, клімат, води, ґрунти, рослинний і тваринний світ)>, <Sentence: До суспільно-географічних (соціально-економічних) належать науки, що вивчають географічне розміщення виробництва, умови та особливості його розвитку в різних країнах та районах — економічна, політична, географія населення, галузей економіки (промисловості, сільського господарства, 

In [157]:
import pandas as pd

In [324]:
en_summaries_df = pd.read_csv('./datasets/en.csv')
en_summaries_df.tail()

,title,text,random_summary,luhn_summary,kl_summary,reduction_summary,text_rank_summary,lex_rank_summary,lsa_summary
95,Azerbaijan,"Azerbaijan (, ; , ), officially the Republic o...",The region and seven surrounding districts are...,"Azerbaijan (, ; , ), officially the Republic o...",Baku is the capital and largest city. The regi...,"Azerbaijan (, ; , ), officially the Republic o...","Azerbaijan (, ; , ), officially the Republic o...",It is a part of the South Caucasus region and ...,It is a part of the South Caucasus region and ...
96,Amateur astronomy,Amateur astronomy is a hobby where participant...,Even though scientific research may not be the...,Amateur astronomers typically view the sky at ...,Even though scientific research may not be the...,Amateur astronomy is a hobby where participant...,Amateur astronomy is a hobby where participant...,"Most amateurs are hobbyists, while others have...",Amateur astronomy is a hobby where participant...
97,Aikido,"Aikido ( , , , ) is a modern Japanese martial...","Aikido ( , , , ) is a modern Japanese martial...","Aikido ( , , , ) is a modern Japanese martial...","Aikido ( , , , ) is a modern Japanese martial...","Aikido ( , , , ) is a modern Japanese martial...","Aikido ( , , , ) is a modern Japanese martial...",It was originally developed by Morihei Ueshiba...,It was originally developed by Morihei Ueshiba...
98,Art,"Art is a diverse range of human activity, and ...","In the Western tradition, the three classical ...",There is no generally agreed definition of wha...,"Art is a diverse range of human activity, and ...","Theatre, dance, and other performing arts, as ...","Theatre, dance, and other performing arts, as ...","Art is a diverse range of human activity, and ...","Art is a diverse range of human activity, and ..."
99,Agnostida,Agnostida are an order of extinct arthropods w...,"Regardless, they appear to be close relatives ...",Agnostida are an order of extinct arthropods w...,"Regardless, they appear to be close relatives ...",Agnostida are an order of extinct arthropods w...,Agnostida are an order of extinct arthropods w...,Agnostida are an order of extinct arthropods w...,Agnostida are an order of extinct arthropods w...


In [325]:
en_summaries_df = en_summaries_df.dropna()

In [326]:
en_summaries_df.shape

(93, 9)

In [322]:
uk_summaries_df = pd.read_csv('./datasets/uk.csv')
uk_summaries_df.tail()

,title,text,random_summary,luhn_summary,kl_summary,reduction_summary,text_rank_summary,lex_rank_summary,lsa_summary
95,Технологія,"Техноло́гія (від , що походить від ; — майсте...","Техноло́гія (від , що походить від ; — майсте...",") про способи (набір і послідовність операцій,...",має декілька базисних значень. Технологія — су...,Технологія — сукупність методів (способів) виг...,Технологія — сукупність методів (способів) виг...,") про способи (набір і послідовність операцій,...",") про способи (набір і послідовність операцій,..."
96,Інтернет,"Інтерне́т (від ), міжнародна комп'ютерна мереж...","Інтерне́т (від ), міжнародна комп'ютерна мереж...","Інтерне́т (від ), міжнародна комп'ютерна мереж...","Інтернет також називають мережею мереж, бо він...","Інтерне́т (від ), міжнародна комп'ютерна мереж...","Інтерне́т (від ), міжнародна комп'ютерна мереж...","Інтерне́т (від ), міжнародна комп'ютерна мереж...",Міжмережжя становить фізичну основу для розміщ...
97,База даних,"База даних () – сукупність даних, організовани...","Таким чином, сучасна база даних, крім самих да...","База даних () – сукупність даних, організовани...","База даних () – сукупність даних, організовани...","База даних () – сукупність даних, організовани...","База даних () – сукупність даних, організовани...",Дані у базі організовують відповідно до моделі...,"В загальному випадку база даних містить схеми,..."
98,Музика,Му́зика (від — мистецтво муз) — мистецтво орг...,Пізнавальна функція — пов'язана із природним п...,Музика як вид мистецтва З точки зору класифіка...,Музичним може бути практично будь-який звук з ...,Му́зика (від — мистецтво муз) — мистецтво орг...,Му́зика (від — мистецтво муз) — мистецтво орг...,Джерелами такого звуку можуть бути: людський г...,) виконавським мистецтвом (посередником між тв...
99,Шевченко Тарас Григорович,"Тара́с Григо́рович Шевче́нко (, с. Моринці, Ки...","Моринці, Київська губернія, Російська імперія ...","Моринці, Київська губернія, Російська імперія ...","Тара́с Григо́рович Шевче́нко (, с. Національни...",Тарас Шевченко народився 9 березня 1814 року в...,Тарас Шевченко народився 9 березня 1814 року в...,"Тара́с Григо́рович Шевче́нко (, с. Діяч україн...","Літературна спадщина Шевченка, центральну роль..."


In [323]:
uk_summaries_df = uk_summaries_df.dropna()
uk_summaries_df.shape

(100, 9)

### SummaQA Evaluation

In [150]:
from summaqa import QG_masked
from summaqa import QA_Metric

In [237]:
def generate_summa_qa_scores(article, summaries, target_df):
    question_generator = QG_masked()
    masked_questions, answer_spans = question_generator.get_questions(article)

    for summary_name, summary in summaries.items():
        qa_metric = QA_Metric()
        score = qa_metric.compute(masked_questions, answer_spans, summary)

        target_df.at[summary_name, 'summa_qa_avg_prob'] = score['avg_prob']
        target_df.at[summary_name, 'summa_qa_avg_fscore'] = score['avg_fscore']

### Blanc Evaluation

In [153]:
from blanc import BlancHelp, BlancTune

In [241]:
def generate_blank_scores(article, summaries, target_df):
    blanc_help = BlancHelp()
    blanc_tune = BlancTune(finetune_mask_evenly=False, finetune_batch_size=24)

    documents = [article]
    summary_texts = [summaries.values]

    help = blanc_help.eval_summaries_for_docs(documents, summary_texts)
    tune = blanc_tune.eval_summaries_for_docs(documents, summary_texts)

    for index, summary_name in enumerate(summaries.keys()):
        target_df.at[summary_name, 'blanc_help'] = help[0][index]
        target_df.at[summary_name, 'blanc_tune'] = tune[0][index]

### Evaluation DataFrame

In [328]:
def evaluate_summaries(summaries_df_series, lang_name):
    # check file existence

    print(f"Evaluating summaries for {summaries_df_series.title}")
    summary_names = en_summaries_df.columns[2:]

    summary_evaluation_df = pd.DataFrame(
        index=summary_names,
        columns=["summa_qa_avg_prob", "summa_qa_avg_fscore", "blanc_help", "blanc_tune"]
    )

    # generate_summa_qa_scores(summaries_df_series.text, summaries_df_series.iloc[2:], summary_evaluation_df)
    generate_blank_scores(summaries_df_series.text, summaries_df_series.iloc[2:], summary_evaluation_df)
    summary_evaluation_df.to_csv(f'./evaluations/{lang_name}/{summaries_df_series.title}.csv')

In [ ]:
for i, series in uk_summaries_df.iloc[:10].iterrows():
    evaluate_summaries(series, 'uk')

Evaluating summaries for Географія


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model t